Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0

# Train GDN Model with BATADAL data

## Table of Contents

1. Load processed data
  * Load sensor columns
  * Load train and test CSVs
2. Load model and environment config files
3. Train GDN model 
4. Perform inference on test set

In [ ]:
import sys 
sys.path.append('../../src/anomaly_detection_spatial_temporal_data/')

In [ ]:
import pandas as pd
import numpy as np
import yaml
from model.GDN.GDNTrainer import GDNTrainer

# Load data

### Load sensor columns

In [ ]:
data_dir = "../../data/03_primary/iot"

In [ ]:
with open(f"{data_dir}/iot_sensor_list_batadal.txt", "r") as f:
    sensors = f.read().split("\n")

In [ ]:
print(f"Number of sensors: {len(sensors)}")

In [ ]:
print(sensors)

### Load train and test CSVs

In [ ]:
train_df = pd.read_csv(f"{data_dir}/iot_gdn_train.csv")
test_df = pd.read_csv(f"{data_dir}/iot_gdn_test.csv")

print(train_df.shape)
print(test_df.shape)


### Load training and environment configurations

In [ ]:
model_config_file = "../../conf/base/parameters/gdn.yml"

In [ ]:
with open(model_config_file, "r") as stream:
    try:
        model_config = yaml.safe_load(stream)
        print(model_config)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
train_config = model_config["train_config"]
env_config = model_config["env_config_wifi"]

env_config["checkpoint_save_dir"] = "../../data/07_model_output/gdn-iot-notebook"

# Train model

In [ ]:
# by default this runs for 3 epochs
# we can change this by uncommenting the following line before creating the training object
# train_config["epoch"] = 5

trainer = GDNTrainer(
    sensors, train_df, test_df, 
    train_config, env_config
)

In [ ]:
trainer.run()

# Run inference

In [ ]:
pred, labels = trainer.predict()

In [ ]:
# we lose 5 items due to the windowing process from the TimeDataset constructor
# window is `slide_win`
# pred.shape

In [ ]:
# np.array(labels).shape

# References
Riccardo Taormina and Stefano Galelli and Nils Ole Tippenhauer and Elad Salomons and Avi Ostfeld and Demetrios G. Eliades and Mohsen Aghashahi and Raanju Sundararajan and Mohsen Pourahmadi and M. Katherine Banks and B. M. Brentan and Enrique Campbell and G. Lima and D. Manzi and D. Ayala-Cabrera and M. Herrera and I. Montalvo and J. Izquierdo and E. Luvizotto and Sarin E. Chandy and Amin Rasekh and Zachary A. Barker and Bruce Campbell and M. Ehsan Shafiee and Marcio Giacomoni and Nikolaos Gatsis and Ahmad Taha and Ahmed A. Abokifa and Kelsey Haddad and Cynthia S. Lo and Pratim Biswas and M. Fayzul K. Pasha and Bijay Kc and Saravanakumar Lakshmanan Somasundaram and Mashor Housh and Ziv Ohar; "The Battle Of The Attack Detection Algorithms: Disclosing Cyber Attacks On Water Distribution Networks." Journal of Water Resources Planning and Management, 144 (8), August 2018

Ailin Deng and Bryan Hooi. 2021. Graph Neural Network-Based Anomaly Detection in Multivariate Time Series. CoRR abs/2106.06947, (2021). Retrieved from https://arxiv.org/abs/2106.06947 